# Rutina para descargar series temporales

# 1. Configuración inicial
Esta celda importa las bibliotecas necesarias y define variables de configuración.

python
Copiar código


In [1]:
import os
import requests
import xarray as xr

# Configuración
base_url = r"https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf"
output_dir = r"./chirps_data"
os.makedirs(output_dir, exist_ok=True)  # Crear el directorio de salida si no existe


# 2. Función para descargar archivos
Incluye una función reutilizable para descargar datos desde la URL.

In [2]:
# Función para descargar un archivo desde una URL
def download_chirps_file(url, output_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Descarga completada: {output_path}")
        return True
    else:
        print(f"Error al descargar {url}. Código de estado: {response.status_code}")
        return False


# 3. Función para procesar archivos NetCDF
Esta celda define la función para cargar y visualizar datos de los archivos NetCDF.


In [3]:
# Función para procesar un archivo NetCDF
def process_chirps_data(nc_file):
    if not os.path.exists(nc_file):
        print(f"Archivo no encontrado: {nc_file}")
        return None
    try:
        data = xr.open_dataset(nc_file)
        print(data)  # Imprime la estructura del dataset
        return data
    except Exception as e:
        print(f"Error al procesar el archivo {nc_file}: {e}")
        return None


# 4. Descargar y combinar la serie histórica
Esta celda descarga los archivos por año y combina la información en un único archivo o estructura para evitar múltiples peticiones.

In [4]:
# Lista de años disponibles (ajusta según necesidad)
years = list(range(1981, 2023))  # CHIRPS tiene datos desde 1981 hasta el presente

# Descargar y combinar archivos NetCDF
combined_data = None

years = list(range(1981, 2023))  # Ajusta los años según los datos disponibles

for year in years:
    filename = f"chirps-v2.0.{year}.daily.nc"  # Cambiado a datos diarios
    url = f"{base_url}/{filename}"
    output_path = os.path.join(output_dir, filename)

    # Descargar el archivo si no existe
    if not os.path.exists(output_path):
        if download_chirps_file(url, output_path):
            print(f"Archivo {filename} descargado.")
    else:
        print(f"Archivo {filename} ya existe, omitiendo descarga.")

    # Procesar y combinar datos
    data = process_chirps_data(output_path)
    if data is not None:
        combined_data = xr.concat([combined_data, data], dim="time") if combined_data else data

# Guardar el conjunto combinado para uso futuro
combined_file = os.path.join(output_dir, r"chirps_combined.nc")
if combined_data:
    combined_data.to_netcdf(combined_file)
    print(f"Serie histórica combinada guardada en: {combined_file}")


Error al descargar https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/chirps-v2.0.1981.daily.nc. Código de estado: 404
Archivo no encontrado: ./chirps_data\chirps-v2.0.1981.daily.nc
Error al descargar https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/chirps-v2.0.1982.daily.nc. Código de estado: 404
Archivo no encontrado: ./chirps_data\chirps-v2.0.1982.daily.nc
Error al descargar https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/chirps-v2.0.1983.daily.nc. Código de estado: 404
Archivo no encontrado: ./chirps_data\chirps-v2.0.1983.daily.nc
Error al descargar https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/chirps-v2.0.1984.daily.nc. Código de estado: 404
Archivo no encontrado: ./chirps_data\chirps-v2.0.1984.daily.nc
Error al descargar https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/chirps-v2.0.1985.daily.nc. Código de estado: 404
Archivo no encontrado: ./chirps_data\chirps-v2.0.1985.daily.nc
Error al descar

# 5. Cargar y manipular los datos combinados
Esta celda carga la serie histórica ya combinada desde el archivo guardado, lo que evita la necesidad de procesar múltiples archivos.

In [5]:
# Cargar datos combinados desde el archivo guardado
combined_file = os.path.join(output_dir, "chirps_combined.nc")

if os.path.exists(combined_file):
    historical_data = xr.open_dataset(combined_file)
    print(historical_data)
else:
    print("El archivo combinado no existe. Asegúrate de ejecutar la celda anterior.")


El archivo combinado no existe. Asegúrate de ejecutar la celda anterior.


# 6. Ejemplo de manipulación de datos
Esta celda muestra cómo extraer información de la serie histórica combinada.

In [6]:
# Ejemplo: Extraer datos mensuales para una región específica
region_data = historical_data.sel(lat=slice(-10, 10), lon=slice(-60, -40))  # Región (latitud/longitud)
mean_precipitation = region_data['precip'].mean(dim=["lat", "lon"])  # Promedio espacial

# Visualizar datos
mean_precipitation.plot()


NameError: name 'historical_data' is not defined